In [17]:
import pyomo.environ as pe
import pyomo.opt as po

### Problema Car Renting 

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B 

Se plantea como variables binarias. 
xij = 1 o 0 . En cada pais i, cada coche j, elijo o no el coche (variable binaria)

Parámetros: 
- coste de alquiler de cada coche en cada país 

Restricciones (todas las lógicas preposicionales del problema)
- coste de 25€ por cada cambio 
- un coche por país (solo puede valer 1 coche 1, y los otros 2 deben valer 0 )
- empieza en españa y termina en italia 



Función objetivo : minimizar los costes 


In [18]:
modelo = pe.ConcreteModel()

#### Parametros
$i$: paises
$j$: coches

In [19]:
modelo.paises = pe.Set(initialize=['Spain', 'France', 'Germany', 'Austria', 'Switzerland','Italy'])
modelo.coches = pe.Set(initialize=['Car1', 'Car2', 'Car3'])



#### Parametros

$c_{ij}$: coste de combustible de, en cada pais i, el tipo de coche j $j$ [€/unit] 

In [20]:
combustible_dict = {
    ('Spain','Car1'):160, ('France','Car1'):210, ('Germany','Car1'):180, ('Austria','Car1'):110, ('Switzerland','Car1'):85,  ('Italy','Car1'):170,
    ('Spain','Car2'):120, ('France','Car2'):240, ('Germany','Car2'):165, ('Austria','Car2'):135, ('Switzerland','Car2'):100, ('Italy','Car2'):160,
    ('Spain','Car3'):150, ('France','Car3'):200, ('Germany','Car3'):175, ('Austria','Car3'):140, ('Switzerland','Car3'):115, ('Italy','Car3'):135,
}
modelo.combustible_coche_pais = pe.Param(modelo.paises, modelo.coches, initialize=combustible_dict)
modelo.coste_cambio = pe.Param(initialize=25)


#### Variables

$coche_{ij}$: 1 si se ha elegido el coche $j$ en el pais $i$ , 0 en caso contrario[bool]


In [21]:

modelo.coche_pais = pe.Var(modelo.paises, modelo.coches, within=pe.Binary)

$cambio_{i}$: 1 si al entrar en el pais $i$ se cambia de coche respecto al pais anterior, 0 en caso contrario [bool]

In [22]:
#variable binaria que vale 1 si se ha cambiado de coche entre el pais anterior y este y 0 en caso contrario
modelo.cambio_pais = pe.Var(['France', 'Germany', 'Austria', 'Switzerland','Italy'], within=pe.Binary)

#### Objective Function

min $\sum_{i} \sum_{j}(c_{ij} * coche_{ij} ) + \sum_{i}(C *cambio_{i} )$



In [23]:
def funcion_objetivo(modelo):
    #queremos minimizar la suma del combustible gastado y los costes de cambio de coche

    # el combustible = combustible por coche por pais * si se ha elegido ese coche en ese pais
    combustible = sum(modelo.combustible_coche_pais[i, k] * modelo.coche_pais[i, k] for i in modelo.paises for k in modelo.coches)

    #cambios = coste de cambio * si se ha cambiado de coche en ese pais
    cambios = sum(modelo.coste_cambio * modelo.cambio_pais[i] for i in modelo.cambio_pais)
    return combustible + cambios

modelo.coste = pe.Objective(rule=funcion_objetivo, sense=pe.minimize)


RESTRICCIONES

Restricción 1: Solo puede haber un coche por país

$\sum_{j}x_{ij}=1 \quad \forall i$

In [24]:
#restriccion de un solo coche por pais
def un_coche_por_pais(modelo, i):
    return sum(modelo.coche_pais[i, k] for k in modelo.coches) == 1

modelo.AsignacionUnica = pe.Constraint(modelo.paises, rule=un_coche_por_pais)

Restricción 2: El cambio de coche es la diferencia entre si se cambia antre el coche anterior y este. 

$cambio_{i} \ge coche_{ij} - coche_{i-1, j} \quad$

In [25]:
#restriccion de cambio de coche
diccionario_paises_anterior = {'France':'Spain', 'Germany':'France', 'Austria':'Germany', 'Switzerland':'Austria', 'Italy':'Switzerland'}
# aplicamos la restriccion coche_pais[i,k]-coche_pais[i,k-1]>=0 

def cambio_mayor_igual(modelo, coche, pais):
    # Solo desde el 2º país (donde existe cambio_pais[i])
    if pais not in modelo.cambio_pais:
        return pe.Constraint.Skip
    # Detecta 0→1; con asignación única por país, basta para detectar el cambio
    return modelo.cambio_pais[pais] >= modelo.coche_pais[pais, coche] - modelo.coche_pais[diccionario_paises_anterior[pais], coche]

modelo.cambio_mayor_igual = pe.Constraint(modelo.coches, modelo.paises, rule=cambio_mayor_igual)


Resolvemos el problema 

In [26]:
solver = po.SolverFactory('gurobi')
resultado = solver.solve(modelo)

# Mostrar estado de la optimización
print ("Estado de la optimización:", resultado.solver.status)
print ("Condición de terminación:", resultado.solver.termination_condition)



Estado de la optimización: ok
Condición de terminación: optimal


In [27]:
print("Coste optimo:", pe.value(modelo.coste))

Coste optimo: 890.0


In [28]:
# Mostrar la asignación de coches por país
for i in modelo.paises:
    for k in modelo.coches:
        if pe.value(modelo.coche_pais[i, k]) > 0.5:  # Si la variable es 1 (o muy cercana a 1)
            print(f"En {i} se alquila el {k}")

En Spain se alquila el Car2
En France se alquila el Car1
En Germany se alquila el Car1
En Austria se alquila el Car1
En Switzerland se alquila el Car1
En Italy se alquila el Car3


In [29]:
modelo.pprint()

2 Set Declarations
    coches : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Car1', 'Car2', 'Car3'}
    paises : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Spain', 'France', 'Germany', 'Austria', 'Switzerland', 'Italy'}

2 Param Declarations
    combustible_coche_pais : Size=18, Index=paises*coches, Domain=Any, Default=None, Mutable=False
        Key                     : Value
            ('Austria', 'Car1') :   110
            ('Austria', 'Car2') :   135
            ('Austria', 'Car3') :   140
             ('France', 'Car1') :   210
             ('France', 'Car2') :   240
             ('France', 'Car3') :   200
            ('Germany', 'Car1') :   180
            ('Germany', 'Car2') :   165
            ('Germany', 'Car3') :   175
              ('Italy', 'Car1') :   170
              ('Italy', 'Car2') :   160
              (

In [30]:
modelo.display()

Model unknown

  Variables:
    coche_pais : Size=18, Index=paises*coches
        Key                     : Lower : Value : Upper : Fixed : Stale : Domain
            ('Austria', 'Car1') :     0 :   1.0 :     1 : False : False : Binary
            ('Austria', 'Car2') :     0 :  -0.0 :     1 : False : False : Binary
            ('Austria', 'Car3') :     0 :  -0.0 :     1 : False : False : Binary
             ('France', 'Car1') :     0 :   1.0 :     1 : False : False : Binary
             ('France', 'Car2') :     0 :  -0.0 :     1 : False : False : Binary
             ('France', 'Car3') :     0 :  -0.0 :     1 : False : False : Binary
            ('Germany', 'Car1') :     0 :   1.0 :     1 : False : False : Binary
            ('Germany', 'Car2') :     0 :  -0.0 :     1 : False : False : Binary
            ('Germany', 'Car3') :     0 :  -0.0 :     1 : False : False : Binary
              ('Italy', 'Car1') :     0 :   0.0 :     1 : False : False : Binary
              ('Italy', 'Car2') :  